<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module4/M4_1/3_Fixed_VariableCosts_Mayhugh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integer Programming: Fixed and Variable Costs
**OPIM 5641: Business Decision Modeling - University of Connecticut**

Please refer to Powell Chapter 11 for more details and examples.

-----------------------------------------------------------------------

# Fixed and Variable Costs

One of the assumptions in linear models is strict proportionality: The cost contributed by
an activity is proportional to its activity level. 

However, we commonly encounter situations
in which activity costs are composed of fixed costs and variable costs, with only the variable
costs being proportional to activity level. With an integer programming model, we can also
integrate the fixed component of cost.
Imagine that we have already built a linear programming model, but one variable ($x$)
has a fixed cost that we want to represent in the objective function. To incorporate this
fixed cost into the model, we separate the fixed and variable components of cost. In
algebraic terms, we write cost as
$\text{Cost} = Fy + cx$
where $F$ represents the fixed cost, and $c$ represents the linear variable cost. The
variables $x$ and $y$ are decision variables, where $x$ is a normal (continuous) variable and
$y$ is a binary variable. (See figure below.)

![Fixed and Variable Costs](https://drive.google.com/uc?export=view&id=1bdo_DGdhlVh8UyljqfnN3ROM9yMc_0M6)

In [ ]:
%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

# Mayhugh Manufacturing
Mayhugh Manufacturing, a medium-size job shop, has been producing and selling three product families. Each product family requires production hours in each of three departments. In addition, each product family requires its own sales force, which must be supported no matter how large or small the sales volume happens to be. The parameters describing the situation are summarized in the following tables:

**Table:** Manufacturing time per product and total hours available

. | F1 | F2 | F3| Hours Available
---|---|---|---|---|
Department A| 3| 4| 8| 2000
Department B| 3 | 5| 6| 2000
Department C| 2| 3| 9| 2000

**Table:** Sales cost, product demand, profit per unit

. | F1 | F2 | F3|
---|---|---|---|
Sales cost (\$000)| 60 | 200| 100
Demand (000)| 400| 300| 50| 
Profit per unit| \$1.20| \$1.80| \$2.20


The fixed cost are the sales costs (you need them no matter what) - the variable costs are how many widgets (F1, F2, F3) to create.




# Scenario 1: You keep all product lines and ignore fixed costs.
As a first attempt, let's treat this as a simple product-mix problem (and just add the fixed costs later on). We are trying to maximize the profit based on how many units of $F_1$, $F_2$ and $F_3$ we are making - and just like previous examples, we have to efficiently use all of the manufacturing hours available to us while not exceeding demand.

**Objective function (given in 000's):**

$\max(z) = 1.2F_1 + 1.8F_2 + 2.2F_3 - 360$

The $-360$ comes from the fixed costs associated with the dedicated salesforce for each product like $F_1$ (60K), $F_2$ (200K), and $F_3$ (100K).


**Constraints (manufacturing hours):**

$3F_1 + 4F_2 + 8F_3 \leq 2000$ 

$3F_1 + 5F_2 + 6F_3 \leq 2000$ 

$2F_1 + 3F_2 + 9F_3 \leq 2000$ 

**Constraints (product demand):**

$F_1 \leq 400$

$F_2 \leq 300$

$F_3 \leq 50$

We have a lot of experience with this type of problem. The code is presented below.

In [ ]:
# 1) concrete model
model = ConcreteModel()

# 2) declare decision variables
model.F1 = Var(domain=NonNegativeIntegers) # number of iphones (integer)
model.F2 = Var(domain=NonNegativeIntegers) # number of ipods
model.F3 = Var(domain=NonNegativeIntegers) # number of laptops

# 3) write the objective function: we are maximizing profit
# don't forget to subtract your fixed costs!
# obj func coeff is simply profit contribution F1, F2, F3
# we are simply substracting fixed costs at the end (NO DECISION VARIABLE)
model.OBJ = Objective(expr = 1.2*model.F1 + 1.8*model.F2 + 2.2*model.F3 - 360,
                      sense=maximize)

# 4) constraints
# manufacturing constraints
model.Constraint1 = Constraint(expr = 3*model.F1 + 4*model.F2 + 8*model.F3 <= 2000) # dept A
model.Constraint2 = Constraint(expr = 3*model.F1 + 5*model.F2 + 6*model.F3 <= 2000) # dept B
model.Constraint3 = Constraint(expr = 2*model.F1 + 3*model.F2 + 9*model.F3 <= 2000) # dept C

# demand constraints
model.Constraint4 = Constraint(expr = model.F1 <= 400) # demand for F1
model.Constraint5 = Constraint(expr = model.F2 <= 300)
model.Constraint6 = Constraint(expr = model.F3 <= 50)

# 5) check your work, then solve!
model.pprint()

3 Var Declarations
    F1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    F2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    F3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 1.2*F1 + 1.8*F2 + 2.2*F3 - 360

6 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body               : Upper  : Active
        None :  -Inf : 3*F1 + 4*F2 + 8*F3 : 2000.0 :   True
    Constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body               : Upper  : Active

Solve the model!

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 410.0
  Upper bound: 410.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of binary variables: 0
  Number of integer variables: 3
  Number of nonzeros: 3
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
    

Show the results. So we will make a profit of 410 (K) and will manufacture 400 units of $F_1$, 100 units of $F_2$ and 50 units of $F_3$.

In [ ]:
# show the results
print("Total profit = ", model.OBJ())
print('F1 units = ', model.F1())
print('F2 units = ', model.F2())
print('F3 units = ', model.F3())

Total profit =  410.0
F1 units =  400.0
F2 units =  100.0
F3 units =  50.0


And if you want to check on constraints, just print them off. Some constraints are binding. 

In [ ]:
print('Manufacturing Constraint A: ', model.Constraint1()) # binding
print('Manufacturing Constraint B: ', model.Constraint2()) # binding
print('Manufacturing Constraint C: ', model.Constraint3()) # not
print('Demand Constraint F1: ', model.Constraint4()) # binding
print('Demand Constraint F2: ', model.Constraint5()) # not
print('Demand Constraint F3 ', model.Constraint6()) # not

Manufacturing Constraint A:  2000.0
Manufacturing Constraint B:  2000.0
Manufacturing Constraint C:  1550.0
Demand Constraint F1:  400.0
Demand Constraint F2:  100.0
Demand Constraint F3  50.0


This is one way of solving the problem - implicitly assume that you just have a product mix problem and then subtract a constant (your fixed costs, 360K). 

But we can so much better! Look at the next few scenarios.

# Scenario 2: Consider dropping product line(s)
Before we talk about dropping product lines, we will need to introduce linking constraints.

# Linking Constraints
 Constraints in the linear program involve
only the variable portion—that is, they involve only the variable $x$, not the variable $y$.
In this situation, we also want to make sure that the variables $x$ and $y$ work together
consistently. In particular, we want to have $y=1$ (so that we incur the fixed cost) when
$x=0$, and we want to have $y=0$ (so that we avoid the fixed cost) when $x=0$. To achieve *consistent* linking of the two variables, we add the following generic linking constraint to the model:

<center>
$x \leq My$
</center>

where the number $M$ represents an upper bound on the variable $x$. In other words, $M$
is at least as large as any value we can feasibly choose for $x$ (like a maximum demand).


Why does the linking constraint work? It is just another feasibility condition to be satisfied. When $y=0$ (and therefore no fixed cost is
incurred), the right-hand side becomes zero, and the constraint is interpreted as $x=0$. 

Since we also require $x=0$, these two constraints together force $x$ to be zero. Thus, when $y=0$, it will be consistent to avoid the fixed cost. On the other hand, when $y=1$, the right-hand side will be so large that Solver does not need to restrict $x$ at all,
permitting its value to be positive while we incur the fixed cost. Thus, when $y=1$, it will be consistent to incur the fixed cost. Of course, because we are optimizing, Solver will never produce a solution with the combination of $y=1$ and $x=0$ because it would
always be preferable to set $y=0$.


In the previous scenario, there was no basis for determining whether any one of the families should be
dropped, because fixed cost considerations are not within the scope of the linear programming analysis.
To formulate the full problem as an integer programming model, we make two changes. 

First, we write the objective function with separate terms for variable profit and
fixed cost (we'll get rid of F1, F2 and F3 and instead introduce a subscript $_1$ for F1 etc.):

$\max(\text{NetProfit}) = 1.2x_1 - 60y_1 + 1.8x_2 - 200y_2 + 2.2x_3 - 100y_3$

Right off the bat, the above formula wouldn't fixed much, other than that we've interested $y$ for binary decision variables and $x$ for our continuous variables. Our naming convention will be such that $y$ is ALWAYS for binary decisions (fixed costs) and $x$ is for continuous variables (variable costs). But this is only part of the story...

In order to ensure consistency, we also introduce these linking constraints.

<center>
$x_1 - M_1y_1 \leq 0$

$x_2 - M_2y_2 \leq 0$

$x_3 - M_3y_3 \leq 0$
</center>

Each of the above constraints will ensure that we do not make any $x_1$, $x_2$ or $x_3$ if we don't engage in that product line!!! This is AWESOME. Such a simple hack - yet it ensures that we won't need those fixed costs if we aren't going into that product line.

The key is ensuring you pick an appropriate value for $M$'s. In our case, demand makes a lot of sense (400 for $F_1$, 300 for $F_2$ and 100 for $F_3$) - we should not be making more widgets than demand!

Now look at how we can reformulate this streamlined problem all together. What you will find is that the incorporation of a linking constraint can alter your final product mix substantially (and increase profits - at the expense of losing some dedicated sales teams...)


**Objective function:**

$\max(\text{NetProfit}) = 1.2x_1 - 60y_1 + 1.8x_2 - 200y_2 + 2.2x_3 - 100y_3$

subject to:

**Constraints (manufacturing):**

$3x_1 + 4x_2 + 8x_3 \leq 2000$

$3x_1 + 5x_2 + 6x_3 \leq 2000$

$2x_1 + 3x_2 + 9x_3 \leq 2000$

**Constraints (linking constraints - for fixed and variable costs):**

$x_1 - 400y_1 \leq 0$

$x_2 - 300y_2 \leq 0$

$x_3 - 50y_3 \leq 0$

Let it rip and see what we get!




In [ ]:
# 1) concrete model
model = ConcreteModel()

# 2) declare decision variables (x1 instead F1, etc.)
model.x1 = Var(domain=NonNegativeIntegers)
model.x2 = Var(domain=NonNegativeIntegers)
model.x3 = Var(domain=NonNegativeIntegers)
# now add our binary variables
model.y1 = Var(domain=Binary) # can only take on values of 0/1
model.y2 = Var(domain=Binary)
model.y3 = Var(domain=Binary)

# 3) write the objective function: we are maximizing profit
# now we are rewriting this to account for x1 and y1 etc.
# see how nice this is rather than subtracting a constant?
model.OBJ = Objective(expr = 1.2*model.x1 - 60*model.y1 + #F1
                              1.8*model.x2 - 200*model.y2 + #F2
                               2.2*model.x3 - 100*model.y3, #F3
                      sense=maximize)

# 4) constraints
# manufacturing constraints - similar to before, but x1 instead of F1 etc.
model.Constraint1 = Constraint(expr = 3*model.x1 + 4*model.x2 + 8*model.x3 <= 2000) # dept A
model.Constraint2 = Constraint(expr = 3*model.x1 + 5*model.x2 + 6*model.x3 <= 2000) # dept B
model.Constraint3 = Constraint(expr = 2*model.x1 + 3*model.x2 + 9*model.x3 <= 2000) # dept C

# these previous demand costraints are rewritten as LINKING CONSTRAINTS
# which can help us decide if we should even be in that product line!
model.Constraint4 = Constraint(expr = model.x1 - 400*model.y1 <= 0) 
model.Constraint5 = Constraint(expr = model.x2 - 300*model.y2 <= 0)
model.Constraint6 = Constraint(expr = model.x3 - 50*model.y3 <= 0)

# 5) check your work, then solve!
model.pprint()

6 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    x3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    y1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    

Now solve it!

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 508.0
  Upper bound: 508.0
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 6
  Number of binary variables: 3
  Number of integer variables: 6
  Number of nonzeros: 6
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
  

And show the results.

In [ ]:
# show the results
print("Total profit = ", model.OBJ())
print('X1 units = ', model.x1())
print('Y1 binary = ', model.y1())
print('X2 units = ', model.x2())
print('Y2 binary = ', model.y2())
print('X3 units = ', model.x3())
print('Y3 binary = ', model.y3())

Total profit =  508.0
X1 units =  400.0
Y1 binary =  1.0
X2 units =  160.0
Y2 binary =  1.0
X3 units =  0.0
Y3 binary =  0.0


The optimal solution achieves a net profit of \$508,000. In order to attain this
level of profits, the company must produce family F1 up to its ceiling, produce
family F2 at the level of 160,000, and eliminate production of product family F3. In
other words, the integer programming model detects that family F3 does not cover
its fixed cost, and the company would be better off not producing and selling that
family at all.

**WOAH!** Our company is no longer in the business of making $F_3$... it gets the boot and so does its sales team. Notice how a simple reframing can yield a drastically different result.

Sometimes fixed costs can be a dedicated sales team, but it can be so much more - the specialized equipment needed to create a product line, rent, computers, paper for the office etc. Things that are STATIC regardless of how many units of a widget you are producing.

A big change in this formulation of the problem is that our demand constraint has been rewritten as a linking constraint that includes a binary variable and the demand all in one. The fixed costs are included in the objective function, and now they have been broken out instead of being lumped into a single constant (360).

If you want to look at the linking constraints vs. the manufacturing constraints, we can do so like this:

In [ ]:
print('Manufacturing Constraint A: ', model.Constraint1()) # not
print('Manufacturing Constraint B: ', model.Constraint2()) # binding
print('Manufacturing Constraint C: ', model.Constraint3()) # not
print('Linking Constraint x1: ', model.Constraint4()) # binding
print('Linking Constraint x2: ', model.Constraint5()) # not
print('Linking Constraint x3: ', model.Constraint6()) # binding

Manufacturing Constraint A:  1840.0
Manufacturing Constraint B:  2000.0
Manufacturing Constraint C:  1280.0
Linking Constraint x1:  0.0
Linking Constraint x2:  -140.0
Linking Constraint x3:  0.0


On your own, you could do a sensitivity analysis and see how things would change.

# Scenario 3: Product minimums and threshold levels.
Sometimes we encounter cost schedules that require us to participate at a specified
minimum level. For example, in purchasing, we might be able to qualify for a discounted
price if we buy in quantity. As another example, in manufacturing, a setup is sufficiently
disruptive that the line would be set up to assemble a batch of motors only if we’re making
at least a dozen of them, not merely one or two. These examples illustrate a threshold-level
requirement: a decision variable is either at least as large as a specified minimum, or else
it is zero.
The existence of a threshold level does not directly affect the objective function
of a model, and it can be represented in the constraints with the help of binary
variables. Suppose we have a variable $x$ that is subject to a threshold requirement.
Let $m$ denote the minimum feasible value of $x$ if it is nonzero. Then we can capture
this structure in an integer programming model by including the following pair
of constraints:

<center>
$x - my \geq 0$

$x - My \leq 0$
</center>

where, as before, $M$ is a large number that is greater than or equal to any value $x$ could
feasibly take. To see how these two requirements work, consider the two possibilities
for the binary variable. When $y=1$, the constraints reduce to $m \leq x \leq M$, so that $x$ is
forced to be at or above the threshold level. When $y=0$, the constraints reduce to $x=0$. Thus, the pair $x$ and $y$ will behave consistently.

Let's try another formulation of the problem.

Imagine we want to require that product family
$F_2$ have a production level of at least 250,000 if it is to have a sales force. Since the model
already includes the linking constraint $x_2 - 300y_2 \leq 0$, we need only to add the threshold
constraint $x_2 - 250y_2 \geq 0$, and the search for an $F2$ level will be restricted to production
values between 250 and 300, or zero.

Grab all of the code from the previous section, and now add one more threshold constraint, such that if we go into $F_2$, we must meet manufacturing minimums.

In [ ]:
# 1) concrete model
model = ConcreteModel()

# 2) declare decision variables (x1 instead F1, etc.)
model.x1 = Var(domain=NonNegativeIntegers)
model.x2 = Var(domain=NonNegativeIntegers)
model.x3 = Var(domain=NonNegativeIntegers)
# now add our binary variables
model.y1 = Var(domain=Binary) # can only take on values of 0/1
model.y2 = Var(domain=Binary)
model.y3 = Var(domain=Binary)

# 3) write the objective function: we are maximizing profit
# now we are rewriting this to account for x1 and y1 etc.
# see how nice this is rather than subtracting a constant?
model.OBJ = Objective(expr = 1.2*model.x1 - 60*model.y1 +
                              1.8*model.x2 - 200*model.y2 +
                               2.2*model.x3 - 100*model.y3,
                      sense=maximize)

# 4) constraints
# manufacturing constraints - similar to before, but x1 instead of F1 etc.
model.Constraint1 = Constraint(expr = 3*model.x1 + 4*model.x2 + 8*model.x3 <= 2000) # dept A
model.Constraint2 = Constraint(expr = 3*model.x1 + 5*model.x2 + 6*model.x3 <= 2000) # dept B
model.Constraint3 = Constraint(expr = 2*model.x1 + 3*model.x2 + 9*model.x3 <= 2000) # dept C

# these previous demand costraints are rewritten as LINKING CONSTRAINTS
# which can help us decide if we should even be in that product line!
model.Constraint4 = Constraint(expr = model.x1 - 400*model.y1 <= 0) #M
model.Constraint5 = Constraint(expr = model.x2 - 300*model.y2 <= 0)
model.Constraint6 = Constraint(expr = model.x3 - 50*model.y3 <= 0)

# NEW: threshold constraint
model.Constraint7 = Constraint(expr = model.x2 - 250*model.y2 >= 0) #m (make at least 250)

# 5) check your work, then solve!
model.pprint()

6 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    x3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    y1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    

Now solve it!

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 490.0
  Upper bound: 490.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 6
  Number of binary variables: 3
  Number of integer variables: 6
  Number of nonzeros: 6
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.02
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
  

And show the results.

In [ ]:
# show the results
print("Total profit = ", model.OBJ())
print('X1 units = ', model.x1())
print('Y1 binary = ', model.y1())
print('X2 units = ', model.x2())
print('Y2 binary = ', model.y2())
print('X3 units = ', model.x3())
print('Y3 binary = ', model.y3())

Total profit =  490.0
X1 units =  250.0
Y1 binary =  1.0
X2 units =  250.0
Y2 binary =  1.0
X3 units =  0.0
Y3 binary =  0.0


In [ ]:
print('Manufacturing Constraint A: ', model.Constraint1()) # not
print('Manufacturing Constraint B: ', model.Constraint2()) # binding
print('Manufacturing Constraint C: ', model.Constraint3()) # not
print('Linking Constraint x1: ', model.Constraint4()) # binding
print('Linking Constraint x2: ', model.Constraint5()) # not
print('Linking Constraint x3: ', model.Constraint6()) # binding
print('Threshold constraint x2: ', model.Constraint7()) # (x2 - 250y2 >= 0)

Manufacturing Constraint A:  1750.0
Manufacturing Constraint B:  2000.0
Manufacturing Constraint C:  1250.0
Linking Constraint x1:  -150.0
Linking Constraint x2:  -50.0
Linking Constraint x3:  0.0
Threshold constraint x2:  0.0


# Summary - look how far you've come
Let's tabulate all of these results and compare.

Scenario | F1 | F2| F3 | Profit
---|---|---|---|---
Scenario 1| 400 | 100| 50| 410
Scenario 2| 400|160|0|508
Scenario 3|250|250|0|490

Your decisions have real consequences! Whenever you add more constraints to the model, you will never improve the objective function (like moving from Scenario 2 to 3 and requiring product minimums.)